In [1]:
import sys
import torch as t
sys.path.append("../../")
from periodic_padding import periodic_padding
import random

## grid splitting with reflective boundary neighborhood

In [2]:
def split_grids(batch:t.Tensor, kernel_width: int, kernel_height: int) -> t.Tensor:
    unfold_transform = t.nn.Unfold(kernel_size=(kernel_height,kernel_width))
    reflective_padding = t.nn.ReflectionPad2d(1)
    
    padded_batch = reflective_padding(batch).float()
    padded_batch = padded_batch.unsqueeze(1)
    
    return t.transpose(unfold_transform(padded_batch), dim0=1, dim1=2)

In [3]:
test1 = t.arange(64).reshape(1,8,8).float()
split1 = split_grids(test1, kernel_width=4, kernel_height=4)
assert split1.shape == (1, 49, 16)
assert t.all(split1[0,0] == t.tensor([
    9,  8,  9,  10,
    1,  0,  1,  2,
    9,  8,  9,  10,
    17, 16, 17, 18
])) 

test2 = t.arange(32**2).reshape(1,32,32).float()
split2 = split_grids(test2, kernel_width=6, kernel_height=6)
assert split2.shape == (1, 841, 36)
assert t.all(split2[0,0] == t.tensor([
    33,  32,  33,  34,  35,  36,
    1,   0,   1,   2,   3,   4,
    33,  32,  33,  34,  35,  36,
    65,  64,  65,  66,  67,  68,
    97,  96,  97,  98,  99,  100,
    129, 128, 129, 130, 131, 132
]))

## converting tile indices of the checkerboard to index in the unfolded conv list

In [4]:
def tile_idx2conv_idx(row, col, tile_size, batch_size):
    _, batch_height, batch_width = batch_size
    tile_height, tile_width = tile_size
    tile_width -= 2
    tile_height -= 2
    tiles_per_row = batch_width-tile_width+1
    return int(tile_height*row*tiles_per_row + tile_width*col)

In [6]:
test1 = t.arange(64).reshape(1,8,8).float()
split1 = split_grids(test1, kernel_width=4, kernel_height=4)
idx = tile_idx2conv_idx(1,0,(4,4),test1.shape)
assert idx == 14
assert t.all(split1[0,idx] == t.tensor([
    9,  8,  9,  10,
    17, 16, 17, 18,
    25, 24, 25, 26,
    33, 32, 33, 34
]))

## converting on-tile coordinates to on-grid coordinates

In [7]:
def tile_coords2grid_coords(tile_idx, on_tile_coords, grid_dim, tile_dim):
    # coordinates of the pixels on the tiles
    tile_y, tile_x = on_tile_coords
    # coordinates of the tiles in the checkerboard scheme
    tile_rows = tile_idx[:,0]
    tile_cols = tile_idx[:,1]
    # dimensionalities of a single tile
    tile_height, tile_width = tile_dim
    _, grid_height, grid_width = grid_dim
    
    grid_x = tile_cols*tile_width+(tile_x-1)
    grid_y = tile_rows*tile_height+(tile_y-1)
    
    grid_x[grid_x == -1] = 1
    grid_x[grid_x == grid_width] = grid_width-2
    
    grid_y[grid_y == -1] = 1
    grid_y[grid_y == grid_height] = grid_height-2
    
    return t.vstack((grid_y, grid_x)).T

In [8]:
test1 = t.arange(64).reshape(1,8,8).float()
on_tile_coords = (t.tensor((1,1,1,1)), t.tensor((1,1,1,1)))
on_grid_tile_idxs = t.tensor([
    [0,0],
    [0,2],
    [2,0],
    [2,2]
])
result_coords = tile_coords2grid_coords(on_grid_tile_idxs, on_tile_coords, grid_dim=test1.shape, tile_dim=(2,2))

# first set of coordinates should be the top-left pixel in the top-left tile
assert t.all(result_coords[0] == t.tensor([0,0]))
# second set of coordinates should be the top-left pixel of the 3. tile in the first row
assert t.all(result_coords[1] == t.tensor([0,4]))
# third set of coordinates should be the top-left pixel of the 1. tile in the 3. row of the checkerboard
assert t.all(result_coords[2] == t.tensor([4,0]))
# fourth set of coordinates should be top-left pixel of the 3. tile in the 3. row of the checkerboard
assert t.all(result_coords[3] == t.tensor([4,4]))

on_tile_coords = (t.tensor((1,1,1,1)), t.tensor((2,2,2,2)))
on_grid_tile_idxs = t.tensor([
    [0,1],
    [0,3],
    [2,1],
    [2,3]
])
result_coords = tile_coords2grid_coords(on_grid_tile_idxs, on_tile_coords, grid_dim=test1.shape, tile_dim=(2,2))

# first set of coordinates should be the top-right pixel of the 2. tile in the 1. row 
assert t.all(result_coords[0] == t.tensor([0,3]))
# second set of coordinates should be the top-right pixel of the 4. tile in the first row
assert t.all(result_coords[1] == t.tensor([0,7]))
# third set of coordinates should be the top-right pixel of the 2. tile in the 3. row
assert t.all(result_coords[2] == t.tensor([4,3]))
# fourth set of coordinates should be the top-right pixel of the 4. tile in the 3. row
assert t.all(result_coords[3] == t.tensor([4,7]))

on_tile_coords = (t.tensor((0,0,1,3)), t.tensor((1,0,0,2)))
on_grid_tile_idxs = t.tensor([
    [0,0],
    [0,0],
    [0,0],
    [3,3]
])
result_coords = tile_coords2grid_coords(on_grid_tile_idxs, on_tile_coords, grid_dim=test1.shape, tile_dim=(2,2))

# checking if the reflective boundary works as expected. Because of the padding of the tiles, setting
# the on-tile row index to zero would bring us out of bounds. Instead, we want to be reflected back to row 2
assert t.all(result_coords[0] == t.tensor([1,0]))
# checking other boundary conditions
assert t.all(result_coords[1] == t.tensor([1,1]))
assert t.all(result_coords[2] == t.tensor([0,1]))
assert t.all(result_coords[3] == t.tensor([6,7]))

## creating independent checkerboard sets

In [9]:
def indep_chkboard_sets(batch, tile_height, tile_width):
    _, batch_height, batch_width = batch.shape
    tiles_per_row = batch_width//(tile_width-2)
    tiles_per_col = batch_height//(tile_height-2)
    
    assert tiles_per_row % 2 == 0, "please provide a tile_width that allows for even tiling of the row"
    assert tiles_per_col % 2 == 0, "please provide a tile_height that allows for even tiling of the columns"
    
    meshgrid_coords = []
    for row_start_idx in range(2):
        for col_start_idx in range(2):
            row_idxs = t.tensor(range(row_start_idx, tiles_per_row,2))
            col_idxs = t.tensor(range(col_start_idx, tiles_per_col,2))
            meshgrid_coords.append(t.meshgrid((row_idxs, col_idxs), indexing="ij"))
    return meshgrid_coords
     

In [10]:
checkerboard_sets = indep_chkboard_sets(test1, tile_height=4, tile_width=4)

## Update probability function

In [11]:
def p_update(tiles, cur_vol, target_vol, temperature, src_coords, tgt_coords):
    batch_size, _, _ = tiles.shape 
    src_x, src_y = src_coords
    tgt_x, tgt_y = tgt_coords
    vol_changes = (-1 * tiles[range(batch_size), tgt_y, tgt_x]) + tiles[range(batch_size), src_y, src_x]
    total_vol_change = t.sum(vol_changes)
    print(f"vol change: {total_vol_change}")
    adjusted_vol = cur_vol + total_vol_change
    p_update = t.tensor(0.)
    if adjusted_vol <= 2 and adjusted_vol > 0:
        p_update += t.exp(-(target_vol - adjusted_vol)**2/temperature)
    return p_update

## MCS algorithm 

In [25]:
def MCS(batch, checkerboard_sets, target_vol, temperature):
    _, batch_height, batch_width = batch.shape
    vol_kernel = t.tensor([[[
        [0., 0., 0.],
        [0., 1., 0.],
        [0., 0., 0.]
    ]]])
    
    steps = []
    for c, chkboard_set in enumerate(checkerboard_sets):
        #print(f"-------------- Set {c} ---------------------")
        tile_idxs = t.tensor([(chkboard_set[0][i,j], chkboard_set[1][i,j]) for i in range(chkboard_set[0].shape[0]) for j in range(chkboard_set[1].shape[0])])
        #print(f"indices of the tiles on the grid: \n {tile_idxs}")
        padded_batch = t.nn.ReflectionPad2d(1)(batch).float()
        cur_vol = t.sum(t.nn.functional.conv2d(padded_batch, vol_kernel))
        conv_stack = split_grids(batch, kernel_width=4, kernel_height=4)
        n_rows, n_cols = chkboard_set[0].shape
        conv_stack_idxs = [
            tile_idx2conv_idx(
                chkboard_set[0][i,j],
                chkboard_set[1][i,j],
                tile_size=(4,4),
                batch_size=batch.shape
            ) for i in range(n_rows) for j in range(n_cols)
        ]
        #print(f"indices of the tiles in the conv stack: \n {conv_stack_idxs}")
        tiles = t.vstack([conv_stack[0, conv_stack_idxs]]).reshape(len(conv_stack_idxs),4,4)
        src_x = t.randint_like(t.zeros(tiles.shape[0],), low=1, high=3)
        src_x = src_x.type(t.long)
        src_y = t.randint_like(src_x, low=1, high=3)
        # For each random sample in src, we sample a random value from [-1, 0, 1]
        # and add it on to the src_idx
        step_sizes = t.tensor(random.choices([
            (1, 0),
            (1, 1),
            (0, 1),
            (-1, 1),
            (-1, 0),
            (-1, -1),
            (0, -1),
            (1, -1)
        ], k=tiles.shape[0]))
        tgt_x = src_x + step_sizes[:, 1]
        tgt_y = src_y + step_sizes[:, 0]

        grid_coords_src = tile_coords2grid_coords(
            tile_idx=tile_idxs,
            on_tile_coords=(src_y, src_x),
            grid_dim=batch.shape,
            tile_dim=(2,2)
        )
        #print(f"coordinates of the source pixels on the grid: \n {grid_coords_src}")
        #print(f"source pixels: {batch[0, grid_coords_src[:,0], grid_coords_src[:,1]]}")

        grid_coords_tgt = tile_coords2grid_coords(
            tile_idx=tile_idxs,
            on_tile_coords=(tgt_y, tgt_x),
            grid_dim=batch.shape,
            tile_dim=(2,2)
        )

        if t.all(batch[0, grid_coords_src[:,0], grid_coords_src[:,1]] == batch[0, grid_coords_tgt[:,0], grid_coords_tgt[:,1]]):
            print("no match")
        else:
            #print("on_tile source pixel coords per grid: \n",t.cat((src_x.unsqueeze(0), src_y.unsqueeze(0)), dim=0).T)
            #print("on_tile target pixel coords per grid: \n", t.cat((tgt_x.unsqueeze(0), tgt_y.unsqueeze(0)), dim=0).T)
            
            update_probability = p_update(
                tiles,
                cur_vol,
                target_vol,
                temperature,
                (src_x, src_y),
                (tgt_x, tgt_y)
            )
            #print(f"update probability: {update_probability}")
            
            residual = t.rand_like(update_probability)
            
            
            
            #print(f"coordinates of the target pixels on the grid: \n {grid_coords_tgt}")
            #print(f"target pixels: {batch[0, grid_coords_tgt[:,0], grid_coords_tgt[:,1]]}")
            
            
            if update_probability > residual:
                print("update success")
                # update the batch here
                batch[0, grid_coords_tgt[:,0], grid_coords_tgt[:,1]] *= 0
                batch[0, grid_coords_tgt[:,0], grid_coords_tgt[:,1]] += batch[0, grid_coords_src[:,0], grid_coords_src[:,1]]

            steps.append(batch.detach().squeeze().clone().numpy())
    
    return batch, steps
            
        
    

In [26]:
batch, steps = MCS(test1, checkerboard_sets, 1, 1)

vol change: 8.0
vol change: -4.0
vol change: -7.0
vol change: 24.0


In [27]:
test = t.zeros(1,8,8)
test[0,3,3] = 1
checkerboard_sets = indep_chkboard_sets(test1, tile_height=4, tile_width=4)
target_vol = 1.
temperature = 27.


test, steps = MCS(test, checkerboard_sets, target_vol, temperature)
test

no match
no match
vol change: -1.0
no match


tensor([[[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]]])

In [28]:
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image

In [30]:
checkerboard_sets

[(tensor([[0, 0],
          [2, 2]]),
  tensor([[0, 2],
          [0, 2]])),
 (tensor([[0, 0],
          [2, 2]]),
  tensor([[1, 3],
          [1, 3]])),
 (tensor([[1, 1],
          [3, 3]]),
  tensor([[0, 2],
          [0, 2]])),
 (tensor([[1, 1],
          [3, 3]]),
  tensor([[1, 3],
          [1, 3]]))]

In [36]:
test = t.zeros(1,32,32)
test[0,16,16] = 1
checkerboard_sets = indep_chkboard_sets(test, tile_height=4, tile_width=4)
target_vol = 1.
temperature = 27.

states = [test.detach().clone().squeeze().numpy()]
for i in tqdm(range(1000)):
    test, steps = MCS(test, checkerboard_sets, target_vol, temperature)
    states += steps
    if t.sum(test) == 0 or t.sum(test) > 2:
        print(test)
        break

imgs = [Image.fromarray((1-state)*255) for state in states]
imgs[0].save("t_27_reflective.gif", save_all=True, append_images=imgs[1:], duration=10, loop=100)

  1%|          | 6/1000 [00:00<00:17, 56.59it/s]

vol change: 1.0
update success
vol change: -1.0
update success
no match
no match
no match
no match
no match
no match
no match
no match
no match
vol change: -1.0
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
vol change: 1.0
update success
no match
no match
vol change: -1.0
update success
no match
no match
no match
no match
no match
no match
vol change: 1.0
update success
no match
no match
no match
no match
no match
no match
no match
no match
no match
vol change: -1.0
update success
no match
no match


  2%|▏         | 22/1000 [00:00<00:13, 70.99it/s]

no match
no match
no match
no match
vol change: 1.0
update success
no match
vol change: 1.0
no match
no match
no match
no match
vol change: -1.0
update success
vol change: -1.0
no match
no match
no match
vol change: 1.0
update success
no match
no match
no match
vol change: 1.0
no match
vol change: -1.0
update success
no match
no match
no match
no match
no match
no match
no match
no match
no match
vol change: 1.0
update success
vol change: -1.0
update success
no match
no match
no match
no match
vol change: -1.0
no match
vol change: -1.0
no match
no match
no match
no match
vol change: -1.0
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
vol change: 1.0
update success
no match
no match
no match


  4%|▍         | 38/1000 [00:00<00:13, 74.00it/s]

...

no match
no match
vol change: 1.0
no match
vol change: -1.0
update success
no match
no match
no match
no match
no match
vol change: 1.0
no match
no match
no match
no match
no match
no match
no match
vol change: -1.0
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
vol change: 1.0
vol change: -1.0
no match
no match
no match
no match
no match
no match
vol change: 1.0
update success
no match
no match
no match
vol change: 1.0
no match
vol change: -1.0
update success
no match
no match
no match
vol change: -1.0
no match
no match
no match
no match
no match
no match
no match
no match
no match
vol change: -1.0
no match
no match


 99%|█████████▉| 994/1000 [00:13<00:00, 80.92it/s]

no match
no match
no match
no match
no match
no match
no match
vol change: -1.0
no match
no match
no match
no match
vol change: -1.0
no match
no match
no match
no match
vol change: -1.0
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
vol change: -1.0
no match
no match
no match
no match
no match
no match
no match
vol change: -1.0
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
vol change: 1.0
update success
no match
no match
vol change: -1.0
update success
no match
no match
no match
no match
no match
no match


100%|██████████| 1000/1000 [00:13<00:00, 74.43it/s]


no match
no match
vol change: 1.0
update success
no match
no match
no match
no match
vol change: -1.0
update success
no match
no match
vol change: -1.0
no match
no match
no match


In [24]:
len(steps)

0

In [25]:
test = t.zeros(1,8,8)
test[0,3,3] = 1
test[0,4,4] = 1
(test == 1).nonzero(as_tuple=False)

tensor([[0, 3, 3],
        [0, 4, 4]])

In [31]:
def centroid(batch):
    pixel_idxs = (batch == 1).nonzero(as_tuple=False)
    x_mean = t.sum(pixel_idxs[:,1]) / pixel_idxs.shape[0]
    y_mean = t.sum(pixel_idxs[:,2]) / pixel_idxs.shape[0]
    return x_mean, y_mean

In [36]:
test = t.zeros(1,8,8)
test[0,3,3] = 1
test[0,4,4] = 1
cen = centroid(test)

In [37]:
def euclidean_dist(centroid1, centroid2):
    c1_x, c1_y = centroid1
    c2_x, c2_y = centroid2

    return t.sqrt((c1_x - c2_x)**2 + (c1_y - c2_y)**2)

In [41]:
print(euclidean_dist(cen,cen))
print(euclidean_dist(cen,(2.5,2.5)))

tensor(0.)


tensor(1.4142)